# Task 1

## Question 1

In [ ]:
import pandas as pd
import requests

In [3]:
## Assumption: The file is not given and needs to read from the URL

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)

# Make sure the request was successful
if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")

In [4]:
# Read the Excel file
country_df = pd.read_excel('Country-Code.xlsx')


In [12]:
restuarant_list = []

for page in data:
    for restaurant in page['restaurants']:
        restuarant_list.append(restaurant)

In [16]:
def get_details(restuarant_list):
    result = []
    
    for entry in restuarant_list:
        restaurant = entry.get('restaurant', {})
        
        # Get the restaurant id
        restaurant_id = restaurant.get('R', {}).get('res_id', None)
        
        # Get the restaurant name
        restaurant_name = restaurant.get('name', None)
        
        # Get the country_id
        country_id = restaurant.get('location', {}).get('country_id', None)
        
        # Get the city name
        city = restaurant.get('location', {}).get('city', None)
        
        # Get the User Rating Votes
        votes = restaurant.get('user_rating', {}).get('votes', None)
        
        # Get the User Aggregate Rating
        aggregate_rating = restaurant.get('user_rating', {}).get('aggregate_rating', None)
        
        # Get the Cuisines
        cuisines = restaurant.get('cuisines', None)
        
        result.append([restaurant_id, restaurant_name, country_id, city, votes, aggregate_rating, cuisines])
    
    return result
        

In [22]:
columns = ['Restaurant Id', 'Restaurant Name', 'Country', 'City', 'User Rating Votes', 'User Aggregate Rating', 'Cuisines']

restuarant_df = pd.DataFrame(get_details(restuarant_list), columns=columns)

In [26]:
restuarant_df['Country'] = restuarant_df['Country'].map(country_df.set_index('Country Code')['Country'])

In [28]:
restuarant_df.to_csv("restaurants.csv")

## Question 2

In [47]:
events_columns = ['Event Id', 'Restaurant Id', 'Restaurant Name', 'Photo URL', 'Event Title', 'Event Start Date', 'Event End Date']


In [43]:
def get_events(restuarant_list):
    result = []
    
    for entry in restuarant_list:
        restaurant = entry.get('restaurant', {})
        
        restaurant_id = restaurant.get('R', {}).get('res_id', None)
        
        restaurant_name = restaurant.get('name', None)
        
        zomato_events = restaurant.get('zomato_events', [])
        
        for item in zomato_events:
            event = item.get('event', {})
            
            event_id = event.get('event_id', None)
            
            photos = event.get('photos', [])
            
            photo_url = None
            
            for photo in photos:
                photo_url = photo.get('photo', {}).get('url', None)
            
            event_title = event.get('title', None)
            
            event_start_date = event.get('start_date', None)
            
            event_end_date = event.get('end_date', None)
            
            result.append([event_id, restaurant_id, restaurant_name, photo_url, event_title, event_start_date, event_end_date])
    
    return result
        

In [48]:
event_df = pd.DataFrame(get_events(restuarant_list), columns=events_columns)
event_df


,Event Id,Restaurant Id,Restaurant Name,Photo URL,Event Title,Event Start Date,Event End Date
0,322331,18649486,The Drunken Botanist,https://b.zmtcdn.com/data/zomato_events/photos...,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28
1,332812,308322,Hauz Khas Social,https://b.zmtcdn.com/data/zomato_events/photos...,Live 20/20 Match Screenings,2019-03-29,2019-05-23
2,336644,18856789,AIR- An Ivory Region,https://b.zmtcdn.com/data/zomato_events/photos...,Dhol Bhangra Night,2019-04-10,2019-04-11
3,336889,18382360,Local,None,Cocktail Wednesday,2019-04-10,2019-04-10
4,336888,18382360,Local,None,Cocktail Wednesday,2019-04-10,2019-04-10
...,...,...,...,...,...,...,...
179,305850,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Smoky Thursdays,2019-03-01,2019-04-30
180,298162,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Chocolate Tuesdays,2019-04-05,2019-08-21
181,329300,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Review us on Zomato,2019-03-22,2019-04-30
182,329301,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Opening Promo,2019-03-22,2019-04-30
